In [ ]:
import json

# 先建立 dict 格式的 account.json 檔，並取得其中的帳密資料
file = open("account.json", "r", encoding="utf-8")
strJson = file.read()
print(strJson)
dictJson = json.loads(strJson)
print(dictJson['username'])
print(dictJson['password'])

In [ ]:
'''
匯入套件
'''
# 操作 browser 的 API
from selenium import webdriver

# 處理逾時例外的工具
from selenium.common.exceptions import TimeoutException

# 面對動態網頁，等待某個元素出現的工具，通常與 exptected_conditions 搭配
from selenium.webdriver.support.ui import WebDriverWait

# 搭配 WebDriverWait 使用，對元素狀態的一種期待條件，若條件發生，則等待結束，往下一行執行
from selenium.webdriver.support import expected_conditions as EC

# 期待元素出現要透過什麼方式指定，通常與 EC、WebDriverWait 一起使用
from selenium.webdriver.common.by import By

# 強制等待 (執行期間休息一下)
from time import sleep

# 整理 json 使用的工具
import json

# 執行 command 的時候用的
import os

# 引入 hashlib 模組
import hashlib

# 引入 regular expression 工具
import re


'''
Selenium with Python 中文翻譯文檔
參考網頁：https://selenium-python-zh.readthedocs.io/en/latest/index.html
selenium 啓動 Chrome 的進階配置參數
參考網址：https://stackoverflow.max-everyday.com/2019/12/selenium-chrome-options/
Mouse Hover Action in Selenium
參考網址：https://www.toolsqa.com/selenium-webdriver/mouse-hover-action/
'''
# 啟動瀏覽器工具的選項
options = webdriver.ChromeOptions()
# options.add_argument("--headless")                #不開啟實體瀏覽器背景執行
options.add_argument("--start-maximized")         #最大化視窗
options.add_argument("--incognito")               #開啟無痕模式
options.add_argument("--disable-popup-blocking ") #禁用彈出攔截

# 使用 Chrome 的 WebDriver (含 options)
driver = webdriver.Chrome( options = options )

# driver.set_window_size(1200, 960) #視窗大小設定 (寬，高)
# driver.maximize_window() #視窗最大化
# driver.minimize_window() #視窗最小化


'''
自訂變數
'''
# 想爬取資料的來源網址
url = 'https://www.instagram.com/alexstrohl/'

# 放置爬取的資料
listData = []

# 放置 ig 每個格子裡面的超連結
listLink = []

# 設定 set 物件，放置影片、圖片連結，協助過濾重複的元素
setTmp = set()

'''
以 function 名稱，作為爬蟲流程
'''
def login():
    try:
        # 前往首頁
        driver.get('https://www.instagram.com');

        # 等待互動元素出現 (這裡用帳號文字欄位)
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located( 
                (By.CSS_SELECTOR, 'input[name="username"]') 
            )
        )

        # 輸入帳號
        driver.find_element(By.CSS_SELECTOR, 'input[name="username"]').send_keys(dictJson['username'])

        # 輸入密碼
        driver.find_element(By.CSS_SELECTOR, 'input[name="password"]').send_keys(dictJson['password'])

        # 強制等待
        sleep(3)

        # 按下登入
        driver.find_element(By.CSS_SELECTOR, 'button[type="submit"].sqdOP.L3NKy.y3zKF').click()
        
        # 強制等待
        sleep(4)
    except TimeoutException:
        print("等待逾時，即將關閉瀏覽器…")
        sleep(3)
        driver.quit()

# 走訪頁面
def visit():
    # 前往指定連結
    driver.get(url);

# 滾動頁面
def scroll():
    try:
        # 等待篩選元素出現
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located( 
                (By.CSS_SELECTOR, "div.Nnq7C.weEfm") 
            )
        )

        # 瀏覽器內部的高度
        innerHeightOfWindow = 0

        # 當前捲動的量(高度)
        totalOffset = 0

        # 在捲動到沒有元素動態產生前，持續捲動
        while totalOffset <= innerHeightOfWindow:
            # 每次移動高度
            totalOffset += 300;

            # 捲動的 js code
            js_scroll = '''(
                function (){{
                    window.scrollTo({{
                        top:{}, 
                        behavior: 'smooth' 
                    }});
                }})();'''.format(totalOffset)

            # 執行 js code
            driver.execute_script(js_scroll)

            # 強制等待
            sleep(1)

            # 透過執行 js 語法來取得捲動後的當前總高度
            innerHeightOfWindow = driver.execute_script('return window.document.documentElement.scrollHeight;');

            # 印出捲動距離
            print("innerHeightOfWindow: {}, totalOffset: {}".format(innerHeightOfWindow, totalOffset))

            # 為了實驗功能，捲動超過一定的距離，就結束程式
            if totalOffset >= 600:
                break
    except TimeoutException:
        print("等待逾時，即將關閉瀏覽器…")
        sleep(3)
        driver.quit()

# 取得每個項目的 url
def getUrl():
    # 取得主要元素的集合
    a_elms = driver.find_elements(By.CSS_SELECTOR, 'div.Nnq7C.weEfm div.v1Nh3.kIKUG._bz0w a')
    
    # 逐一檢視元素
    for index, a in enumerate(a_elms):
        # 測試功能，所以只選三個連結
        if index >= 6:
            break
        
        # 取得圖片連結
        aLink = a.get_attribute('href')
        print("取得網址: {}".format(aLink))
        
        # 放資料到 list 中
        listLink.append(aLink)

# 逐個網頁連結內容進行分析
def parse():
    for aLink in listLink:
        # 清空暫存圖片、影片連結的 set
        setTmp.clear()
        
        # 走訪頁面
        driver.get(aLink)
        
        '''
        取得頁面的 id：
            full_match = re.search(regex, aLink)[0]
            group_1 = re.search(regex, aLink)[1]
        另一種做法：
            matchObj = re.search(regex, aLink)
        範例 1：
            matchObj.group()  : /p/CGpaFHrHV2j/
            matchObj.group(1) : CGpaFHrHV2j 
        範例 2: 
            matchObj[0]  : /p/CGpaFHrHV2j/
            matchObj[1]  : CGpaFHrHV2j 
        '''
        # 取得 ig 連結的 id
        regex = r'\/p\/([a-zA-Z0-9-_]+)\/'
        pageId = re.search(regex, aLink)[1]
        
        print("網頁 ID: {}".format(pageId))
        
        # 強制等待
        sleep(2)
        
        '''
        判斷是否有「向右」按鈕：
            若有，則代表會有多個 li；
            若無，則代表只有一個 li

        備註：
            由於 find_elements 回傳 list 格式，
            所以可以用 len() 來取得元素長度，
            判斷元素是否存在
        '''
        # 當有右箭頭可以按的時候，代表頁面中同時有多張圖片或影片，甚至混合
        if len(driver.find_elements(By.CSS_SELECTOR, "button._6CZji")) > 0:
            # 取得多元素資訊
            while len(driver.find_elements(By.CSS_SELECTOR, "button._6CZji")) > 0:
                # 按下向右按鈕
                driver.find_element(By.CSS_SELECTOR, "button._6CZji").click()

                '''
                將元素裡面的屬性值(例如 src)，視為 value 加入 set 當中
                '''
                li_elms = driver.find_elements(By.CSS_SELECTOR, "li.Ckrof")

                # 檢視當前現有的 li (解決滑動時，前面一部分或後面一部分 li.Ckrof 會消失的問題)
                for li in li_elms:
                    if len(li.find_elements(By.CSS_SELECTOR, "article img.FFVAD")) > 0: # 如果這個 li 裡面有 img
                        # 取得 img 連結
                        imgSrc = li.find_element(By.CSS_SELECTOR, "article img.FFVAD").get_attribute('src')

                        # 加入圖片連結到 set 中
                        setTmp.add(imgSrc)
                    else:
                        pass

                # 強制等待
                sleep(1)
        else:
            # 當沒有右箭頭可以按的時候，代表是獨立的單張圖片或影片
            if len( driver.find_elements(By.CSS_SELECTOR, "article img.FFVAD") ) > 0: # 若是 img
                # 取得 img
                imgSrc = driver.find_element(By.CSS_SELECTOR, "article img.FFVAD").get_attribute('src')

                # 加入圖片連結到 set 中
                setTmp.add(imgSrc)
            else:
                pass

        # 整合此次網頁連結的元素資料
        listData.append({
            "id": pageId,
            "url": aLink,
            "listDL": list(setTmp)
        })
        
# 將 list 存成 json
def saveJson():
    fp = open("ig.json", "w", encoding='utf-8')
    fp.write( json.dumps(listData, ensure_ascii=False) )
    fp.close()
    
# 關閉瀏覽器
def close():
    driver.quit()

# ??????
def download():
    # 開啟 json 檔案
    fp = open("ig.json", "r", encoding='utf-8')
    
    #取得 json 字串
    strJson = fp.read()
    
    # 關閉檔案
    fp.close()
    
    # 建立儲存圖片、影片的資料夾
    folderPath = 'ig'
    if not os.path.exists(folderPath):
        os.makedirs(folderPath)
    
    # 將 json 轉成 list (裡面是 dict 集合)
    listResult = json.loads(strJson)
    
    # 將 listData 底下的 dict 找出來
    for dictObj in listResult:
        # dictObj 底下有個 listDL 屬性，它對應的值是個 list，將 list裡面所有 url 找出來
        for dl_url in dictObj['listDL']:
            print("下載連結: {}".format( dl_url ))
            strFileName = re.search(r'https?:\/\/\S+.\/(\S+\.(jpe?g|mp4))', dl_url)[1]
            os.system('curl "{}" -o ./{}/{}'.format(dl_url, folderPath, strFileName))
            
'''主程式'''
if __name__ == '__main__':
    login()
    visit()
    scroll()
    getUrl()
    parse()
    saveJson()
    close()
    download()